In [1]:
import tensorflow as tf
import numpy as np
import cv2 as cv
import pandas as pd
from tqdm import tqdm

from datetime import datetime
import glob
import math
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
import pandas as pd
from PIL import Image
import random as python_random
#import seaborn as sns
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import auc, accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.utils import shuffle
import sys
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
#from tensorflow.keras.mixed_precision import experimental as mixed_precision
# pip install image-classifiers==1.0.0b1
#from classification_models.tfkeras import Classifiers
# More information about this package can be found at https://github.com/qubvel/classification_models

#from keras.applications.resnet import ResNet50

from keras.applications.vgg16 import VGG16, preprocess_input

seed = 2022
np.random.seed(seed)
python_random.seed(seed)
tf.random.set_seed(seed)


In [2]:
df_meta = pd.read_csv('data\mimic_iv_csv\df_race_3class_1117.csv')
df_meta

,img_index,subject_id,study_id,dataset,race,gender,age,insurance,survival_year
0,0,11135741,52912807,train,ASIAN,M,53.0,Other,-1.00
1,1,13091743,51959936,train,ASIAN,M,85.0,Medicare,0.85
2,2,13091743,53279329,train,ASIAN,M,85.0,Medicare,0.51
3,3,13091743,53691912,train,ASIAN,M,85.0,Medicare,0.20
4,4,13091743,56124505,train,ASIAN,M,85.0,Medicare,0.68
...,...,...,...,...,...,...,...,...,...
58983,59869,19948788,54253734,test,BLACK,F,44.0,Medicaid,-1.00
58984,59870,19948788,56226482,test,BLACK,F,44.0,Medicaid,-1.00
58985,59871,19948788,58554921,test,BLACK,F,44.0,Medicaid,-1.00
58986,59872,19948788,58554921,test,BLACK,F,44.0,Medicaid,-1.00


In [16]:
df_meta['dataset'].value_counts()

train    35635
val      11715
test     11638
Name: dataset, dtype: int64

In [3]:
train_size, test_size, val_size = df_meta['dataset'].value_counts()

In [27]:
# label_interest = 'race'
# #num_classes = 3
# y_train = df_meta[df_meta['dataset'] == 'train'][label_interest].tolist()
# y_test = df_meta[df_meta['dataset'] == 'test'][label_interest].tolist()
# y_val = df_meta[df_meta['dataset'] == 'val'][label_interest].tolist()

# from sklearn.preprocessing import LabelBinarizer
# encoder = LabelBinarizer()
# y_train = encoder.fit_transform(y_train)
# y_test = encoder.fit_transform(y_test)
# y_val = encoder.fit_transform(y_val)

In [12]:
# # load data
# loaded = np.load('data\X_race_split_1116.npz')
# X_train = loaded['train']
# X_test = loaded['test']
# X_val = loaded['val']

In [4]:
HEIGHT = 256
WIDTH = 256

In [5]:
from keras.applications.resnet import ResNet50, preprocess_input

input_a = Input(shape=(HEIGHT, WIDTH, 3))
base_model = ResNet50(weights='imagenet', 
                    input_tensor=input_a,
                    include_top=False, 
                    input_shape=(HEIGHT, WIDTH, 3)
                    )

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(3, name='dense_logits')(x)
output = Activation('softmax', dtype='float32', name='predictions')(x)
model = Model(inputs=[input_a], outputs=[output])

: 

: 

In [8]:
learning_rate = 1e-3
momentum_val=0.9
decay_val= 0.0

epochs = 20
train_batch_size = 64 # may need to reduce batch size if OOM error occurs
test_batch_size = 64

reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=2, min_lr=1e-5, verbose=1)

adam_opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=decay_val)
adam_opt = tf.keras.mixed_precision.LossScaleOptimizer(adam_opt)

model.compile(optimizer=adam_opt,
                loss=tf.losses.CategoricalCrossentropy(),
                metrics=[
                    tf.keras.metrics.AUC(curve='ROC', name='ROC-AUC'),
                    tf.keras.metrics.AUC(curve='PR', name='PR-AUC')
                ],
)

: 

: 

In [4]:
train_gen = ImageDataGenerator(
            rotation_range=15,
            fill_mode='constant',
            horizontal_flip=True,
            zoom_range=0.1,
            preprocessing_function=preprocess_input
            )

validate_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [7]:
data_dir = 'data/imgs_race'

train_dir = f'{data_dir}/train'
test_dir = f'{data_dir}/test'
val_dir = f'{data_dir}/val'

train_batches = train_gen.flow_from_directory(
                                directory= train_dir,
                                classes = None, # means automatically infer the label from subdir
                                class_mode = 'categorical', # white, black, asian
                                target_size=(HEIGHT, WIDTH),
                                shuffle=True,
                                seed=seed, 
                                batch_size=train_batch_size
)

validate_batches = validate_gen.flow_from_directory(
                                directory= val_dir,
                                classes = None, # means automatically infer the label from subdir
                                class_mode = 'categorical', # white, black, asian
                                target_size=(HEIGHT, WIDTH),
                                shuffle=False,
                                # seed=seed, 
                                batch_size= test_batch_size
)

NameError: name 'train_batch_size' is not defined

In [34]:
train_epoch = math.ceil(train_size / train_batch_size)
val_epoch = math.ceil(val_size / test_batch_size)
print(train_epoch, val_epoch)

557 182


In [21]:
var_date = datetime.now().strftime("%Y%m%d-%H%M%S")
var_date

'20221117-153648'

In [33]:
model_name = 'ResNet50'
arc_name = f"{model_name}_60-20-20-split_3-race_detection"
model_save_fname = "saved_models/" + str(arc_name) + "_LR-" + str(learning_rate) + "_" + var_date+ f"_epoch_{epoch:03d}_val_loss_{val_loss:.5f}.h5"

ES = EarlyStopping(monitor='val_loss', mode='min', patience=4, restore_best_weights=True)
checkloss = ModelCheckpoint(model_save_fname, 
            monitor='val_loss', mode='min', verbose=1, save_best_only=True, save_weights_only=False)

In [27]:
epoch = 5
val_loss = 0.00023423432
print(f'{epoch:02d} {val_loss:.5f}')

05 0.00023


In [31]:
model.fit(train_batches,
            validation_data=validate_batches,
            epochs=epochs,
            steps_per_epoch=int(train_epoch),
            validation_steps=int(val_epoch),
            #workers=32,
            #max_queue_size=50,
            shuffle=True,
            callbacks=[checkloss, reduce_lr, ES]
           )